<a href="https://colab.research.google.com/github/kridtapon/Momentum-Harmony/blob/main/Momentum_Harmony.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.9 MB/s eta 0:00:00


In [ ]:
pip install --upgrade yfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 58.5 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.57
    Uninstalling yfinance-0.2.57:
      Successfully uninstalled yfinance-0.2.57


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Nearly RSX (Relative Strength Xtra)
def rsx(close, length=14):
    """Approximate Jurik RSX indicator"""
    # Constants
    pi = np.pi
    alpha = 1.0 / np.exp(1.0 / length)

    # Initialization
    price = close.values
    f = np.zeros_like(price)
    g = np.zeros_like(price)
    h = np.zeros_like(price)
    j = np.zeros_like(price)
    k = np.zeros_like(price)
    rsx = np.zeros_like(price)

    for i in range(6, len(price)):
        f[i] = price[i] - price[i-1]
        g[i] = f[i] + 0.5 * f[i-1] + 0.33 * f[i-2] + 0.25 * f[i-3]
        h[i] = g[i] - 0.5 * g[i-1] + 0.33 * g[i-2] - 0.25 * g[i-3]
        j[i] = alpha * h[i] + (1 - alpha) * j[i-1]
        k[i] = alpha * j[i] + (1 - alpha) * k[i-1]

        rsx[i] = 50 + (k[i] / (np.abs(k[i]) + 1e-10)) * min(50, np.abs(k[i]) * 100)

    return pd.Series(rsx, index=close.index)

# Function to calculate Know Sure Thing (KST)
def calculate_kst(close):
    roc1 = close.pct_change(10) * 100
    roc2 = close.pct_change(15) * 100
    roc3 = close.pct_change(20) * 100
    roc4 = close.pct_change(30) * 100

    rcma1 = roc1.rolling(10).mean()
    rcma2 = roc2.rolling(10).mean()
    rcma3 = roc3.rolling(10).mean()
    rcma4 = roc4.rolling(15).mean()

    kst = rcma1 + 2 * rcma2 + 3 * rcma3 + 4 * rcma4
    signal = kst.rolling(9).mean()
    return kst, signal

# Download data
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'
df = yf.download(symbol, start=start_date, end=end_date, multi_level_index=False)

# Calculate indicators
df['RSX'] = rsx(df['Close'], length=14)
df['KST'], df['KST_Signal'] = calculate_kst(df['Close'])

# Define Entry and Exit signals
df['Entry'] = (df['RSX'] > 50) & (df['KST'] > df['KST_Signal'])
df['Exit'] = (df['RSX'] < 50) & (df['KST'] < df['KST_Signal'])

# Filter test period
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Shift signals
shifted_entries = df['Entry'].shift(1).fillna(False).astype(bool).to_numpy()
shifted_exits = df['Exit'].shift(1).fillna(False).astype(bool).to_numpy()

# Backtest with VectorBT
portfolio = vbt.Portfolio.from_signals(
    close=df['Open'],
    entries=shifted_entries,
    exits=shifted_exits,
    init_cash=100_000,
    fees=0.001,
    slippage=0.002,
    freq='D'
)

# Output results
print(portfolio.stats())
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
<ipython-input-6-f6281bb2de5b>:66: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  shifted_entries = df['Entry'].shift(1).fillna(False).astype(bool).to_numpy()
<ipython-input-6-f6281bb2de5b>:67: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  shifted_exits = df['Exit'].shift(1).fillna(False).astype(bool).to_numpy()


Start                                2020-01-02 00:00:00
End                                  2024-12-31 00:00:00
Period                                1258 days 00:00:00
Start Value                                     100000.0
End Value                                  137456.635198
Total Return [%]                               37.456635
Benchmark Return [%]                          187.561061
Max Gross Exposure [%]                             100.0
Total Fees Paid                              7718.086489
Max Drawdown [%]                               59.935428
Max Drawdown Duration                 1092 days 00:00:00
Total Trades                                          33
Total Closed Trades                                   33
Total Open Trades                                      0
Open Trade PnL                                       0.0
Win Rate [%]                                   36.363636
Best Trade [%]                                 46.550059
Worst Trade [%]                